# Week 01 Assignment: minimum spanning trees

A minimum spanning tree (MST) of an undirected weighted graph $G=(V,E)$ is a graph $T=(V',E')$ such that
\begin{align*}
V' &= V\\
E' &\subset E
\end{align*}
with $\sum_{e\in E'} w_e$ being the the smallest possible among all choices of $E'$. Here $w_e$ is the weight of an edge.

## Borůvka's algorithm

The basis of all MST algorithm was developed by [Otakar Borůvka](https://en.wikipedia.org/wiki/Otakar_Bor%C5%AFvka) in 1926. Here's the pseudocode.

\begin{align*}
& \textbf{find minimum spanning tree of } G: \\
& \qquad \textbf{initialize}\ T\ \text{as an edgeless copy of}\ G \\
& \qquad \textbf{while}\ T\ \text{has more than 1 components}: \\
& \qquad\qquad \text{connect two components with a safe edge} \\
& \qquad \textbf{return}\ T
\end{align*}


In [56]:
##12345678901234567890123456789012345678901234567890123456789012345678901234567890


class MST:
    """A class with the methods necessary to produce a MST from the adjacency
    matrix of a weighted undirected graph.
    """

    def __init__(self, G: list[list[int]]):
        # Local copy of the input graph's adjacency matrix
        self.G: list[list[int]] = G
        # Shorthand notation for number of vertices
        self.n: int = len(self.G)
        # Shorthand of no-edge sentinel from input graph
        self.no_edge = G[0][0]
        # Adjacency matrix for MST
        self.T: list[list[int]] = [[self.no_edge for i in range(self.n)] for j in range(self.n)] 

    def _reachabilty(self, starting: int, graph: list[list[int]]) -> list[int]:
        """Determines the vertices of the input graph reachable from the
        starting vertex."""
        # List of reachable vertices
        reach: list[int] = []
        # List of vertices to explore next, primed with the starting vertex
        visit_next: list[int] = [starting]
        # Process every vertex in the visit_next list until it's empty.
        while len(visit_next) > 0:
            # Take a vertex out of the visit_next list. This can be the first
            # vertex in the list (operating it as a queue), the last vertex
            # (operating it as a stack), or any vertex with damn well please
            # (operating it as we like). All things being equal, stack is fine.
            vertex: int = visit_next.pop()
            if vertex not in reach:
                # Add this vertex to the reachable list
                reach.append(vertex)
                # Plan to visit this vertex's neighbors next                
                for neighbor in range(self.n):
                    if graph[neighbor][vertex] != self.no_edge:
                        visit_next.append(neighbor)
        return reach  # Done!

    def _count_and_label(self):
        """Counts the components of the candidate MST (self.T) and labels its
        vertices with the component they belong to."""
        count_of_components: int = 0
        # For every vertex in the graph, track its component. The component
        # label is the count value for each component.
        component_label: list[int] = [-1] * self.n
        # Remember the vertices we've seen so far
        visited: list[int] = []
        # Process every vertex
        for vertex in range(self.n):
            if vertex not in visited:
                # We just found a new component, update the count
                count_of_components += 1
                # Find everything we can reach from this vertex, because they'll be
                # in the same component
                reachable_from_vertex = self._reachabilty(vertex, self.T)
                print(reachable_from_vertex)
                # Add these reachable vertices to those visited, because
                # they are all in the same component.
                visited.extend(reachable_from_vertex)
                # The current component count becomes the label of all
                # these vertices
                for v in reachable_from_vertex:
                    component_label[v] = count_of_components
        return count_of_components, component_label  # Done

    def boruvka(self):
        """The algorithm itself!"""
        # Initialize count of components in the candidate MST and also get the
        # labels for each vertex in it.
        count_of_components, component_label = self._count_and_label()
        while count_of_components > 1:
            # Initialize a list to hold the safe edges for the various components.
            # Here lies a YUGE! implementation question. How to represent a safe edge?
            safe_edge = [None] * (self.n + 1)
            # Find any two vertices u, v in different components
            for u in range(self.n):
                for v in range(u + 1, self.n):
                    if component_label[u] != component_label[v]:
                        # Vertices u, v are in different components. Let's figure
                        # out the safe edge for the component of vertex u.
                        if safe_edge[component_label[u]] is None:
                            # There is no safe edge for this component yet, so let's
                            # assume that (u,v) is just it. And here is how to represent
                            # the safe edge of a component.
                            safe_edge[component_label[u]] = [u, v]
                        else:
                            # There is currently a safe edge for this vertex, but is
                            # it truly the safest? Let's find out. Is edge (u,v)
                            # safer than the current safe edge for this component?
                            current_component = component_label[u]
                            current_safe_edge = safe_edge[current_component]
                            a = current_safe_edge[0]  # safe edge vertex a
                            b = current_safe_edge[1]  # safe edge vertex b
                            # Look up input graph for weigh of edge (a,b)
                            current_weight = self.G[a][b]
                            # Which of two edges (a,b) and (u,v) is safer, ie,
                            # has the smallest weight?
                            if self.G[u][v] < current_weight:
                                # If (u,v) is smaller than the existing safe
                                # edge (a,b) make (u,v) the safe edge for the
                                # current component.
                                safe_edge[current_component] = [u, v]
                        # It's component v's turn now. We repeat the same
                        # logic as above, but for component of vertex v. The
                        # code is a bit less verbose this time.
                        if safe_edge[component_label[v]] is None:
                            safe_edge[component_label[v]] = [u, v]
                        else:
                            current_safe_edge = safe_edge[component_label[v]]
                            [a, b] = safe_edge[component_label[v]]
                            current_weight = self.G[a][b]
                            if self.G[u][v] < current_weight:
                                safe_edge[current_component] = [u, v]
            # Add all the safe edges to the candidate MST
            print(safe_edge)
            for edge in safe_edge:
                if edge is not None:
                    u = edge[0]
                    v = edge[1]
                    self.T[u][v] = self.G[u][v]
                    self.T[v][u] = self.G[v][u]
            # Recount the components and re-label the vertices
            count_of_components, component_label = self._count_and_label()
        return self.T  # Done!

In [57]:
# TEST CODE

_ = float("inf")

# The adjacency matrix for the graph used in the examples

graph = [  
    [_, _, _, 5, 1, _],  
    [_, _, 20, 5, _, 10],
    [_, 20, _, 10, _, _],
    [5, 5, 10, _, _, 15],
    [1, _, _, _, _, 20], 
    [_, 10, _, 15, 20, _],
]

mst = MST(graph)
mst.boruvka()
T = (mst.T)
weight = 0
for i in range(len(T)):
    for j in range(i + 1, len(T)):
        if T[i][j] != _:
            weight += T[i][j]
print("Weight of MST is", weight)


[0]
[1]
[2]
[3]
[4]
[5]
[None, [0, 4], [1, 5], [2, 3], [3, 5], [4, 5], [0, 5]]
[0, 4, 5, 3, 2, 1]
Weight of MST is 56
